In [19]:
%load_ext autoreload
%autoreload 2
import requests
from configurations.models import models
from models.dumb.dumb_interviewer import DumbInterviewer

def get_cutoff(from_experiment):
    return {'equal': 3}.get(from_experiment, 10)

def print_missing(missing):
    if missing:
        for idx, name in enumerate(sorted(missing)):
            print(f'{idx + 1}. {name}')
    else:
        print('Nothing!')
    print()

model_names = set()
# Change prefix to e.g., greedy, ddpg, to show only those interviewers
experiment = 'equal'
metric = 'ndcg'
model_prefix = 'pop'
num_questions = 10
splits = 4

data = requests.get(f'https://mindreader.tech/spectate/results/{experiment}/{metric}/{get_cutoff(experiment)}').json()

for model in models:
    if not model.startswith(model_prefix):
        continue
    
    model_names.add(model)
    
    if models[model]['class'] != DumbInterviewer:
       model_names.add(f'{model}-rec')

existing_models = set(data.keys())

print(f'Missing entirely from {experiment} ({model_prefix}):')
print_missing(model_names.difference(existing_models))

print(f'Missing first split:')
print_missing({model for model in existing_models if len(data[model]) < num_questions})

print(f'Missing splits:')
def results_len(from_model):
    return sum([len(split) for split in [question for question in data[from_model]]])

print_missing({model for model in existing_models if len(data[model]) == num_questions and results_len(model) < num_questions * splits})

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Missing entirely from equal (pop):
1. pop-knn
2. pop-knn-rec
3. pop-mf
4. pop-mf-rec
5. pop-ppr-linear
6. pop-ppr-linear-collab
7. pop-ppr-linear-collab-rec
8. pop-ppr-linear-joint
9. pop-ppr-linear-joint-rec
10. pop-ppr-linear-kg
11. pop-ppr-linear-kg-rec
12. pop-ppr-linear-rec

Missing first split:
1. fmf
2. fmf-rec

Missing splits:
1. greedy-knn

